In [1]:
from varoptimizer.var_sampling_optimizer import VarSamplingOptimizer

/BS/var-training/work/var/.venv-var/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device ='cuda'
sampling_optimizer = VarSamplingOptimizer(model_depth=16, device=device)


[constructor]  ==== flash_if_available=True (0/16), fused_if_available=True (fusing_add_ln=0/16, fusing_mlp=0/16) ==== 
    [VAR config ] embed_dim=1024, num_heads=16, depth=16, mlp_ratio=4.0
    [drop ratios ] drop_rate=0.0, attn_drop_rate=0.0, drop_path_rate=0.0666667 (tensor([0.0000, 0.0044, 0.0089, 0.0133, 0.0178, 0.0222, 0.0267, 0.0311, 0.0356,
        0.0400, 0.0444, 0.0489, 0.0533, 0.0578, 0.0622, 0.0667]))

[init_weights] VAR with init_std=0.0180422


In [7]:

import torch
from tqdm import tqdm
import numpy as np

class_labels = (980, 980, 437, 22)

initial_noise = 0.1
cfg = 7
top_k = 600 #700
top_p = 0.65 #0.95 #0.95
lr = 0.08
steps_per_size = 100
seed = 42

scale_by_inv_area = False

total_steps = steps_per_size * len(sampling_optimizer.patch_nums)

label_B: torch.LongTensor = torch.tensor(class_labels, device=device)
f_initial = torch.randn([4, 32, 16, 16], device=device) * initial_noise # std=0.5
f = f_initial.clone().detach().requires_grad_(True)

optimizer = torch.optim.Adam([f], lr=lr)

for b in sampling_optimizer.var.blocks: b.attn.kv_caching(False)

for max_next_map_size_index in range(len(sampling_optimizer.patch_nums)):

    for i in tqdm(range(steps_per_size), total=steps_per_size):
        optimizer.zero_grad()
        loss = 0

        loss += sampling_optimizer.get_optimization_loss(f, label_B=label_B, predicted_map_size_index=max_next_map_size_index, batch_size=4, cfg=cfg, top_k=top_k, top_p=top_p, scale_by_inv_area=scale_by_inv_area, seed=seed)

        random_size_index = np.random.randint(0, max_next_map_size_index + 1)
        loss += sampling_optimizer.get_optimization_loss(f, label_B=label_B, predicted_map_size_index=random_size_index, batch_size=4, cfg=cfg, top_k=top_k, top_p=top_p, scale_by_inv_area=scale_by_inv_area, seed=seed)    

        # if i % 100 == 0:
        #     print(f'loss: {loss.item()}')

        step_weight = np.power((steps_per_size - i + 1) / steps_per_size, 1/3)

        loss*= step_weight
        

        #loss*= step_weight
        loss.backward()
        optimizer.step()

100%|██████████| 100/100 [00:10<00:00,  9.11it/s]


In [8]:
import torchvision
import PIL.Image as PImage

def visualise(f_hat):
    image = sampling_optimizer.vae.fhat_to_img(f_hat.detach()).add_(1).mul_(0.5)

    chw = torchvision.utils.make_grid(image, nrow=8, padding=0, pad_value=1.0)
    chw = chw.permute(1, 2, 0).mul_(255).cpu().numpy()
    chw = PImage.fromarray(chw.astype(np.uint8))
    chw.show()

visualise(f)

kf.service.services: KApplicationTrader: mimeType "x-scheme-handler/file" not found
kf.i18n.kuit: "Unknown subcue ':whatsthis,' in UI marker in context {@info:whatsthis, %1 the action's text}."
org.kde.kdegraphics.gwenview.lib: Unresolved raw mime type  "image/x-samsung-srw"
